In [1]:
import pandas as pd
import bson

In [41]:
countries_df = pd.read_csv('countries.csv',sep=',')

In [27]:
countries_df.head()

,id,country_name,location
0,267,Denmark,"(56.26392, 9.501785)"
1,266,Italy,"(41.87194, 12.56738)"
2,244,Taiwan,"(23.69781, 120.960515)"
3,279,USA,"(37.09024, -95.712891)"
4,88,France,"(46.227638, 2.213749)"


In [42]:
country_id_dict = {}

for index in countries_df.index:
    oid = str(f"ObjectId('{bson.objectid.ObjectId()}')")
    country_id_dict[countries_df['id'][index]] = oid
    countries_df['id'][index] = oid

countries_df = countries_df.rename({'id':'_id'}, axis='columns')

/var/folders/jj/6mz63v6n0b5f1hlgmlnwgd7r0000gn/T/ipykernel_73775/3097098386.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries_df['id'][index] = oid


In [43]:
countries_df.head()

,_id,country_name,location
0,ObjectId('65ed77091c693c20303a7edd'),Denmark,"(56.26392, 9.501785)"
1,ObjectId('65ed77091c693c20303a7ede'),Italy,"(41.87194, 12.56738)"
2,ObjectId('65ed77091c693c20303a7edf'),Taiwan,"(23.69781, 120.960515)"
3,ObjectId('65ed77091c693c20303a7ee0'),USA,"(37.09024, -95.712891)"
4,ObjectId('65ed77091c693c20303a7ee1'),France,"(46.227638, 2.213749)"


In [87]:
cities_df = pd.read_csv('cities.csv',sep=',')

In [67]:
cities_df.head()

,id,city_name,country_id,city_location,average_annual_temperature,c40,land_area,gcom
0,58796,Odder Kommune,ObjectId('65ed77091c693c20303a7edd'),"(55.975718, 10.149958)",NaN,False,NaN,False
1,36158,Comune di Napoli,ObjectId('65ed77091c693c20303a7ede'),"(40.8419943, 14.2513831)",NaN,False,NaN,False
2,62855,Egedal Municipality,ObjectId('65ed77091c693c20303a7edd'),"(55.795045, 12.238733)",8.0,False,126.0,False
3,61753,Yilan County,ObjectId('65ed77091c693c20303a7edf'),"(24.702107, 121.73775)",22.3,False,2143.0,False
4,61790,"City of Emeryville, CA",ObjectId('65ed77091c693c20303a7ee0'),"(37.831316, -122.285247)",15.1,False,5.2,False


In [88]:
for index in cities_df.index:
    if(int(cities_df["country_id"][index]) != 0):
        cities_df["country_id"][index] = country_id_dict[cities_df["country_id"][index]]

/var/folders/jj/6mz63v6n0b5f1hlgmlnwgd7r0000gn/T/ipykernel_73775/1859071649.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_df["country_id"][index] = country_id_dict[cities_df["country_id"][index]]


In [181]:
cities_json = json.loads(cities_df.to_json(orient='records'))

In [182]:
emissions_json = json.loads(pd.read_csv('emission.csv',sep=',').to_json(orient='records'))
target_json = json.loads(pd.read_csv('target.csv',sep=',').to_json(orient='records'))
sectors_json = json.loads(pd.read_csv('sectors.csv',sep=',').to_json(orient='records'))
status_json = json.loads(pd.read_csv('emission_status.csv',sep=',').to_json(orient='records'))
pop_json = json.loads(pd.read_csv('pop.csv',sep=',').to_json(orient='records'))
gdp_json = json.loads(pd.read_csv('gdp.csv',sep=',').to_json(orient='records'))

In [183]:
## adds sector field to target

for target in target_json:
    for sector in sectors_json:
        if target["sector_id"] == sector["id"]:
            target["sector"] = sector["sector_name"]
    del target['sector_id']


In [184]:
## Adds status to emissiom

for emission in emissions_json:
    for status in status_json:
        if emission["emission_status_id"] == status["id"]:
            emission["status"] = status["status_name"]
    del emission['emission_status_id']

In [185]:
for city in cities_json:
    # adds targets to city
    city["emissionTargets"] = []
    
    for target in target_json:
        if(target["city_id"] == city["id"]):
            toAdd = target.copy()
            del toAdd["city_id"]
            del toAdd["id"]
            city["emissionTargets"].append(toAdd)
            

In [186]:
for city in cities_json:
    # adds targets to city
    city["emissions"] = []
    
    for em in emissions_json:
        if(em["city_id"] == city["id"]):
            toAdd = em.copy()
            del toAdd["city_id"]
            del toAdd["id"]
            city["emissions"].append(toAdd)
            

In [187]:
for city in cities_json:
    # adds targets to city
    city["gdps"] = []
    
    for gdp in gdp_json:
        if(gdp["city_id"] == city["id"]):
            toAdd = gdp.copy()
            del toAdd["city_id"]
            del toAdd["id"]
            city["gdps"].append(toAdd)

In [188]:
for city in cities_json:
    # adds targets to city
    city["populations"] = []
    
    for pop in pop_json:
        if(pop["city_id"] == city["id"]):
            toAdd = pop.copy()
            del toAdd["city_id"]
            del toAdd["id"]
            city["populations"].append(toAdd)

In [189]:
cities_json[11]

{'id': 1850,
 'city_name': 'Birmingham City Council',
 'country_id': "ObjectId('65ed77091c693c20303a7ee4')",
 'city_location': '(52.480174, -1.902907)',
 'average_annual_temperature': 9.2,
 'c40': False,
 'land_area': 267.8,
 'gcom': True,
 'emissionTargets': [{'Baseline_year': 1990.0,
   'target_year': 2027,
   'baseline_emissions': 6874000.0,
   'percentage_reduction_target': 60.0,
   'sector': 'Total'}],
 'emissions': [{'reporting_year': 2016,
   'measurement_year': 2013,
   'total_emissions': 5199.7,
   'total_scope_1_emissions': None,
   'total_scope_2_emissions': None,
   'status': 'Decreased'}],
 'gdps': [{'gdp': 84000000000.0,
   'currency': 'GBP     Pound Sterling',
   'gdp_year': 2014}],
 'populations': [{'population_amount': 1101360, 'population_year': 2014}]}

In [190]:
for city in cities_json:
    del city["id"]
    city["_id"] = str(f"ObjectId('{bson.objectid.ObjectId()}')")

In [191]:
with open("cities.json", "w") as outfile:
    json.dump(cities_json, outfile)